# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *
from tools_routing import *

In [ ]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load and edit files

LAM output analysis.

simu irr et simu no-irr.

In [ ]:
date_min='2010-01-01'
date_max='2022-12-31'

## Sims


### LMDZ

In [ ]:
TS_flag=False

In [ ]:
# noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/noirr_2010_2022'
# irr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/irr_2010_2022'
noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/noirr'
irr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/irr'

In [ ]:
#sim
if TS_flag:
    filename = '{}/*/ATM/TS_MO/*.nc'.format(noirr_dir)
else:
    filename = '{}/*/ATM/MO/*.nc'.format(noirr_dir)


sim0 = xr.open_mfdataset(filename)
sim0.attrs['name'] = 'no_irr'
sim = sim0.rename({'time_counter':'time'})
sim.attrs["plot_color"] = 'red'
sim = sim.sel(lon=slice(lon_min, lon_max),lat=slice(lat_min, lat_max))

sim['evap'] = sim['evap'] *3600 * 24
sim['evap'].attrs['units'] = 'mm/d'
sim['evap'].attrs['long_name'] = 'Evapotranspiration'

sim['precip'] = sim['precip'] *3600 * 24
sim['precip'].attrs['units'] = 'mm/d'
sim['precip'].attrs['long_name'] = 'Total precipitation'

sim['fluxsens']= -sim['sens']

sim['netrad'] = sim['LWdnSFC'] - sim['LWupSFC'] + sim['SWdnSFC'] - sim['SWupSFC']
sim['netrad'].attrs['units'] = 'W/m2'

sim['SWnetSFC'] = sim['SWdnSFC'] - sim['SWupSFC']
sim['SWnetSFC'].attrs['units'] = 'W/m2'

sim['LWnetSFC'] = sim['LWdnSFC'] - sim['LWupSFC']
sim['LWnetSFC'].attrs['units'] = 'W/m2'

sim['P - E'] = sim['precip'] - sim['evap']
sim['P - E'].attrs['units'] = 'mm/d'

sim['psol'] = sim['psol'] / 100
sim['psol'].attrs['units'] = 'hPa'

sim=compute_grid_cell_width(sim)
sim=add_moisture_divergence(sim)


if not TS_flag :
    sim['Altitude'] = sim['phis'] / 9.81
    sim['Altitude'].attrs['units'] = 'm'

sim

In [ ]:
#sim irr
if TS_flag:
    filename = '{}/*/ATM/TS_MO/*.nc'.format(irr_dir)
else:
    filename = '{}/*/ATM/MO/*.nc'.format(irr_dir)

sim0irr = xr.open_mfdataset(filename)
sim0irr.attrs['name'] = 'irr'
simirr = sim0irr.rename({'time_counter':'time'})
simirr.attrs["plot_color"] = 'blue'
simirr = simirr.sel(lon=slice(lon_min, lon_max),lat=slice(lat_min, lat_max))

simirr['evap'] = simirr['evap'] *3600 * 24
simirr['evap'].attrs['units'] = 'mm/d'

simirr['precip'] = simirr['precip'] *3600 * 24
simirr['precip'].attrs['units'] = 'mm/d'

simirr['fluxsens']= -simirr['sens']

simirr['netrad'] = simirr['LWdnSFC'] - simirr['LWupSFC'] + simirr['SWdnSFC'] - simirr['SWupSFC']
simirr['netrad'].attrs['units'] = 'W/m2'

simirr['SWnetSFC'] = simirr['SWdnSFC'] - simirr['SWupSFC']
simirr['SWnetSFC'].attrs['units'] = 'W/m2'

simirr['LWnetSFC'] = simirr['LWdnSFC'] - simirr['LWupSFC']
simirr['LWnetSFC'].attrs['units'] = 'W/m2'

simirr['P - E'] = simirr['precip'] - simirr['evap']
simirr['P - E'].attrs['units'] = 'mm/d'

simirr['psol'] = simirr['psol'] / 100
simirr['psol'].attrs['units'] = 'hPa'

simirr=compute_grid_cell_width(simirr)
simirr=add_moisture_divergence(simirr)

if not TS_flag:
    simirr['Altitude'] = simirr['phis'] / 9.81
    simirr['Altitude'].attrs['units'] = 'm'

simirr


### ORC

In [ ]:
#open netcdf files
if TS_flag:
    filename = '{}/*/SRF/TS_MO/*.nc'.format(noirr_dir)
elif not TS_flag:
    filename = '{}/*/SRF/MO/*sechiba_history.nc'.format(noirr_dir)

ORCnoirr0 = xr.open_mfdataset(filename)
ORCnoirr = ORCnoirr0.rename({'time_counter':'time'})
ORCnoirr.attrs['name'] = 'no_irr'
ORCnoirr.attrs['plot_color'] = "red"
ORCnoirr = ORCnoirr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

ORCnoirr['aei_sw'] = ORCnoirr['aei_sw'] * 100

ORCnoirr

In [ ]:
if TS_flag:
    filename = '{}/*/SRF/TS_MO/*.nc'.format(noirr_dir)
elif not TS_flag:
    filename = '{}/*/SRF/MO/*history.nc'.format(irr_dir)

ORCirr0 = xr.open_mfdataset(filename)
ORCirr = ORCirr0.rename({'time_counter':'time'})
ORCirr.attrs['name'] = 'irr'
ORCirr.attrs['plot_color'] = "#0C5DA5"
ORCirr = ORCirr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

ORCirr['aei_sw'] = ORCirr['aei_sw'] * 100


ORCirr

In [ ]:
if not TS_flag:
    # #manually define irrig_deficit as netirrig-irrigation in ORC file
    # ORCirr['irrig_deficit'] = ORCirr['netirrig'] - ORCirr['irrigation']
    # #make irrig_deficit units mm/day
    # ORCirr['irrig_deficit'].attrs['units'] = 'mm/day'
    # ORCirr['irrig_frac'] = ORCirr['irrigmap_dyn']/ORCirr['Areas']
    ORCnoirr['irrigmap_dyn'] = ORCirr['irrigmap_dyn']

    ORCnoirr['irrig_frac'] = ORCirr['irrigmap_dyn']/ORCirr['Areas'] * 100
    ORCnoirr['irrig_frac'].attrs['units'] = '%'
    ORCirr['irrig_frac'] = ORCirr['irrigmap_dyn']/ORCirr['Areas'] * 100
    ORCirr['irrig_frac'].attrs['units'] = '%'

### Routing

In [ ]:
rename_dict = {
        'time_counter' : 'time',
        'routing_stream_reservoir_r' : 'streamr',
        'routing_fast_reservoir_r' : 'fastr',
        'routing_slow_reservoir_r' : 'slowr',
        'routing_drainage_r' : 'drainage',
        'routing_runoff_r' : 'runoff',
        'routing_hydrographs_r' : 'hydrographs',
        'routing_irrigation_r' : 'irrigation',
        'irrig_netereq_r' : 'netirrig'
        }

long_name_dict = {
        'streamr' : 'stream reservoir',
        'fastr' : 'fast reservoir',
        'slowr' : 'slow reservoir',
        'drainage' : 'drainage',
        'runoff' : 'runoff',
        'hydrographs' : 'hydrographs',
        'irrigation' : 'irrigation',
        'netirrig' : 'net irrigation'
        }

In [ ]:
# filename = '{}/SRF/MO/sim1*_hydrographs_monthly_*.nc'.format(noirr_dir)
filename = '{}/*/SRF/MO/*diag_routing_r.nc'.format(noirr_dir)
routing_noirr = xr.open_mfdataset(filename)

routing_noirr = routing_noirr.rename(rename_dict)
routing_noirr = routing_noirr.sel(lon=slice(lon_min, lon_max),lat=slice(lat_max, lat_min))

#add long name to variables
for var in routing_noirr.variables:
    if var in long_name_dict.keys():
        routing_noirr[var].attrs['long_name'] = long_name_dict[var]
from dateutil.relativedelta import relativedelta

#move data 1 month back to correct for output file convention
routing_noirr['time'] = routing_noirr['time'].copy(
    data=[(pd.Timestamp(t.item()) - relativedelta(months=1)).to_datetime64() for t in routing_noirr['time'].values]
)

#add unit m³/s to hydrographs
routing_noirr['hydrographs'].attrs['units'] = 'm³/s'

routing_noirr.attrs['name'] = 'noirr'
routing_noirr.attrs['plot_color'] = 'red'

routing_noirr

In [ ]:
filename = '{}/*/SRF/MO/*diag_routing_r.nc'.format(irr_dir)
routing_irr = xr.open_mfdataset(filename)

routing_irr = routing_irr.rename(rename_dict)
routing_irr = routing_irr.sel(lon=slice(lon_min, lon_max),lat=slice(lat_max, lat_min))

#add long name to variables
for var in routing_irr.variables:
    if var in long_name_dict.keys():
        routing_irr[var].attrs['long_name'] = long_name_dict[var]

#move data 1 month back to correct for output file convention
routing_irr['time'] = routing_irr['time'].copy(
    data=[(pd.Timestamp(t.item()) - relativedelta(months=1)).to_datetime64() for t in routing_irr['time'].values]
)

#add unit m³/s to hydrographs
routing_irr['hydrographs'].attrs['units'] = 'm³/s'

routing_irr.attrs['name']='irr'
routing_irr.attrs['plot_color'] = 'blue'

routing_irr

## Obs

### GLEAM

In [ ]:
filename = '../../../obs/GLEAMv4.1a/E/*'
gleam = xr.open_mfdataset(filename)
gleam.attrs['name'] = 'GLEAMv4.1a'
gleam.attrs['plot_color'] = 'black'
#restrict lon lat
gleam = gleam.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max,lat_min))

gleam['evap']=convert_mm_per_month_to_mm_per_day(gleam['E'])
gleam['evap'].attrs['units'] = 'mm/d'
gleam

### GPCC

In [ ]:
#gpcc
filename='../../../obs/precips/precip.mon.total.0.25x0.25.v2020.nc'
gpcc0=xr.open_mfdataset(filename)
gpcc0.attrs['name'] = 'GPCC'
gpcc=gpcc0.sel(time=slice(date_min,date_max))
gpcc['lon'] = ((gpcc['lon'] + 180) % 360) - 180
gpcc1 = gpcc.sortby('lon')

# gpcc=gpcc.sel(lon=slice(-13,6.25),lat=slice(lat_min, lat_max))
gpcc=gpcc1.sel(lon=slice(-13,6.25),lat=slice(lat_max, lat_min))

gpcc.attrs["plot_color"] = 'black'

# gpcc1=gpcc.sel(lon=slice(347,360),lat=slice(lat_max, lat_min))
# gpcc2=gpcc.sel(lon=slice(0,6.25),lat=slice(lat_max, lat_min))
# gpcc=xr.combine_by_coords([gpcc1, gpcc2])
# gpcc = gpcc.where(gpcc['lon'] < 6.25, drop=True)

# gpcc['lon']=gpcc['lon']-180.0
# gpcc=gpcc.sel(lon=slice(-13,6.25))

gpcc['precip'] = convert_mm_per_month_to_mm_per_day(gpcc['precip'])
gpcc['precip'].attrs['units'] = 'mm/d'
gpcc

### Ebro irrig

In [ ]:
load_Ebro_obs=True

In [ ]:
if (load_Ebro_obs):
    filename = '../../../obs/SM_based_inversion_approach_EBRO_Irrigation.nc'
    obsEbro = xr.open_dataset(filename)
    obsEbro = obsEbro.rename({'Time':'time'})
    obsEbro = obsEbro.rename({'Longitude':'lon'})
    obsEbro = obsEbro.rename({'Latitude':'lat'})
    obsEbro = obsEbro.rename({'Irrigation':'irrigation'})
    #make lon and lat coordinates and not variables
    # obsEbro = obsEbro.set_coords(['lon','lat'])

    obsEbro = regrid_to_lon_lat(obsEbro)
    obsEbro.attrs['name'] = 'Ebro_estimate'
    obsEbro.attrs['plot_color'] = "black"

    #convert to mm/day
    obsEbro['irrigation'] = obsEbro['irrigation']/30
    obsEbro['irrigation'].attrs['units'] = 'mm/d'
    obsEbro['irrigation'].attrs['long_name'] = 'Irrigation'
    
    #resample obsEbro to monthly values
    obsEbro = obsEbro.resample(time='1M').mean()
    
    obsEbro

### Discharge

In [ ]:
filename = '../../../obs/streamflow/GRDC-Monthly_Spain.nc'
obs_routing = xr.open_dataset(filename)
obs_routing

## Masking, interpolation, and time period selection

### Sim masks

In [ ]:
#continental fraction mask (LMDZ outputs)
con_mask=sim.mean(dim='time')['contfracOR']>0.95
ip_mask=polygon_to_mask(sim, iberian_peninsula) * con_mask

In [ ]:
ip_sim = sim.where(ip_mask, drop=False)
ip_simirr = simirr.where(ip_mask, drop=False)

In [ ]:
ip_ORCirr=ORCirr.where(ip_mask, drop=False)
ip_ORCnoirr=ORCnoirr.where(ip_mask, drop=False)

### Interpolation and filtering of products

In [ ]:
if (load_Ebro_obs):
    ebro_mask = obsEbro['irrigation']>0
    ORCirr_iObsEbro = ORCirr.interp_like(obsEbro)
    ebro_ORCirr_iObsEbro = ORCirr_iObsEbro.where(ebro_mask)
    obsEbro_iORC = obsEbro.interp_like(ORCirr)
    ebro_mask_iORCirr = obsEbro_iORC['irrigation']>0
    ebro_ORCirr = ORCirr.where(ebro_mask_iORCirr)

In [ ]:
#gleam
gleam_isim=gleam.interp_like(sim)
cont_gleam_isim=gleam_isim.where(con_mask)
ip_gleam_isim = cont_gleam_isim.where(ip_mask, drop=False)

In [ ]:
#gpcc
gpcc_isim=gpcc.interp_like(sim)
cont_gpcc_isim=gpcc_isim.where(con_mask)
ip_gpcc_isim = cont_gpcc_isim.where(ip_mask, drop=False)

# Figures

## Fig 1 : domaine et altitude

In [ ]:
import psyplot.project as psy
psy.rcParams['auto_show'] = True
mpl.rcParams['figure.figsize'] = [8, 6.]

In [ ]:
LAM_dir='../../../JZ_simu_outputs/LAM/native_outputs/'
start_file='{}/espagne_start_2010.nc'.format(LAM_dir)
start=xr.open_dataset(start_file)
start

In [ ]:
start["Altitude"] = start["phis"]/9.81
start["Altitude"].attrs["units"] = "m"
start["Altitude"].attrs["long_name"] = "Altitude"
start["Altitude"].attrs["standard_name"] = "Altitude"

start_file2='{}/espagne_start_2010_modified.nc'.format(LAM_dir)
start.to_netcdf(start_file2)

In [ ]:
start2=xr.open_dataset(start_file2)
start2

In [ ]:
# Custom colormap
terrain_colors = plt.cm.terrain(np.linspace(0, 1, 25))
filtered_colors = terrain_colors[5:]  
# Insert blue as the first color
filtered_colors = np.vstack([[0.07973856209150328, 0.44052287581699345, 0.8405228758169935, 1.0], filtered_colors]) 
custom_colormap = ListedColormap(filtered_colors)
# custom_colormap.set_under('lightblue')  # Define blue for out-of-range low values
custom_colormap

In [ ]:
file=start_file2
var='Altitude'
vmin=-0
vmax=2000
pas=(vmax-vmin) / 20

map=psy.plot.mapplot(
    file, 
    name=var,
    datagrid=dict(color='k', linewidth=0.2),
    cbar='r',
    tight=True,
    lsm='50m',
    cmap=custom_colormap,
    extend='both',
    projection='ortho',
    xgrid=True,ygrid=True,
    bounds=np.arange(vmin, vmax + pas, pas),
    # title=var,
    clabel="Altitude (m)"
)

## Fig 2 : stations et barrages

In [ ]:
proper_stations_dict = {
    6226800: {'name': 'Tortosa',            'river': 'Ebro',            'lat_grid': 40.82500,   'lon_grid': 0.525007,   'station_nb': 1},
    6226400: {'name': 'Zaragoza',           'river': 'Ebro',            'lat_grid': 41.67499,   'lon_grid': -0.90832,   'station_nb': 2},
    6226300: {'name': 'Castejon',           'river': 'Ebro',            'lat_grid': 42.17499,   'lon_grid': -1.69165,   'station_nb': 3},
    6226600: {'name': 'Seros',              'river': 'Segre',           'lat_grid': 41.45833,   'lon_grid': 0.425007,   'station_nb': 4},
    6226650: {'name': 'Fraga',              'river': 'Cinca',           'lat_grid': 41.52499,   'lon_grid': 0.341674,   'station_nb': 5},
    6212410: {'name': 'Tore',               'river': 'Douro',           'lat_grid': 41.50833,   'lon_grid': -5.47499,   'station_nb': 6},
    6212700: {'name': 'Peral De Arlanza',   'river': 'Arlanza',         'lat_grid': 42.07500,   'lon_grid': -4.07499,   'station_nb': 7},
    6213700: {'name': 'Talavera',           'river': 'Tagus',           'lat_grid': 39.95833,   'lon_grid': -4.82499,   'station_nb': 8},
    6213800: {'name': 'Trillo',             'river': 'Tagus',           'lat_grid': 40.70833,   'lon_grid': -2.57499,   'station_nb': 9},
    6213900: {'name': 'Peralejos',          'river': 'Tagus',           'lat_grid': 40.59166,   'lon_grid': -1.92499,   'station_nb': 10},
    6216510: {'name': 'Azud de Badajoz',    'river': 'Guadiana',        'lat_grid': 38.86199,   'lon_grid': -7.01,      'station_nb': 11}, 
    6116200: {'name': 'Pulo do Lobo',       'river': 'Guadiana',        'lat_grid': 37.803,     'lon_grid': -7.633,     'station_nb': 12},         
    6216530: {'name': 'La Cubeta',          'river': 'Guadiana',        'lat_grid': 38.975,     'lon_grid': -2.895,     'station_nb': 13},         
    6216520: {'name': 'Villarubia',         'river': 'Guadiana',        'lat_grid': 39.125,     'lon_grid': -3.59073,   'station_nb': 14},      
    6216800: {'name': 'Quintanar',          'river': 'Giguela',         'lat_grid': 39.64166,   'lon_grid': -3.07499,   'station_nb': 15},
    6217140: {'name': 'Mengibar',           'river': 'Guadalquivir',    'lat_grid': 37.98425,   'lon_grid': -3.79939,   'station_nb': 16},     
    6217200: {'name': 'Arroyo Maria',       'river': 'Guadalquivir',    'lat_grid': 38.17905,   'lon_grid': -2.83594,   'station_nb': 17}, 
    6217700: {'name': 'Pinos Puente',       'river': 'Frailes',         'lat_grid': 37.27499,   'lon_grid': -3.75832,   'station_nb': 18},
}

#keeping only 3 representative stations for 3 larger rivers, for simple figure
#keep only first 3 stations of proper_stations_dict
# representative_stations_dict = dict(list(proper_stations_dict.items())[:5])
representative_stations_dict = {
    6226800: {'name': 'Tortosa',            'river': 'Ebro',            'lat_grid': 40.82500,   'lon_grid': 0.525007,   'station_nb': 1},
    6212410: {'name': 'Tore',               'river': 'Douro',           'lat_grid': 41.50833,   'lon_grid': -5.47499,   'station_nb': 6},
    6213700: {'name': 'Talavera',           'river': 'Tagus',           'lat_grid': 39.95833,   'lon_grid': -4.82499,   'station_nb': 8},
    6216510: {'name': 'Azud de Badajoz',    'river': 'Guadiana',        'lat_grid': 38.86199,   'lon_grid': -7.01,      'station_nb': 11}, 
    6217140: {'name': 'Mengibar',           'river': 'Guadalquivir',    'lat_grid': 37.98425,   'lon_grid': -3.79939,   'station_nb': 16},     
}

In [ ]:
filename='../../../obs/Europe-dams_edited.ods'
dams = pd.read_excel(filename, engine='odf')
mask = dams['Country']=='Spain'
dams = dams[mask]
rename_dict = {
    'Name of dam': 'Name',
    'Decimal degree latitude': 'lat',
    'Decimal degree longitude': 'lon',
    'Reservoir capacity (million m3)' : 'capacity'
}
dams.rename(columns=rename_dict, inplace=True)

#filter dams to keep only the biggest in Reservoir capacity (million m3) 
# dams=dams.nlargest(50, 'capacity')
dams

In [ ]:
stations_map_dict(proper_stations_dict, river_cond=None, legend=True,
                            dams_df=dams, dam_nb=252, title=None,
                            extent=[-10, 2.5, 36, 44]
                            )

## Fig 3 : irrigation maps

In [ ]:
vars=['irrig_frac', 'aei_sw', 'netirrig', 'irrigation', 'slowr', 'streamr']
cmaps=[reds, reds, blues, wet, emb_neutral, emb_neutral]
vmins=[0, 0, 0, 0, -100, -100]
vmaxs=[50, 100, 10, 0.6, 100, 100]
clabels=['Irrigated fraction (%)',
         'Share of surfae withdrawals (%)',
         'Irrigation requirement (mm d⁻¹)',
         'Irrigation (mm d⁻¹)',
         'Groundwater reservoir change (%)',
         'River reservoir change (%)']

fig, axes = plt.subplots(3, 2, figsize=(17,12), subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.flatten()
for i in range(4):
    var=vars[i]
    plotvar=ip_ORCirr[var].mean(dim='time')
    ax=axes[i]
    cmap=cmaps[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    # ax.add_feature(cfeature.RIVERS)
    nice_map(plotvar, ax, cmap, vmin, vmax, clabel=clabel)
    title='({})'.format(letters[i])
    ax.set_title(title)

for i in range(4, 6):
    var=vars[i]
    rel_diff = (ip_ORCirr[var] - ip_ORCnoirr[var]).mean(dim='time') / ip_ORCnoirr[var].mean(dim='time') * 100
    plotvar= rel_diff
    ax=axes[i]
    cmap=cmaps[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    nice_map(plotvar, ax, cmap, vmin, vmax, clabel=clabel)
    #add rivers
    # ax.add_feature(cfeature.RIVERS)
    title='({})'.format(letters[i])
    ax.set_title(title)

In [ ]:
# version with only 4 maps
# vars=['netirrig', 'irrigation', 'slowr', 'streamr']
# cmaps=[blues, wet, emb_neutral, emb_neutral]
# vmins=[ 0, 0, -100, -100]
# vmaxs=[10, 0.6, 100, 100]
# clabels=[
#          'Irrigation requirement (mm d⁻¹)',
#          'Irrigation (mm d⁻¹)',
#          'Groundwater reservoir change (%)',
#          'River reservoir change (%)']

# fig, axes = plt.subplots(2, 2, figsize=(17,8), subplot_kw={'projection': ccrs.PlateCarree()})
# axes = axes.flatten()
# for i in range(2):
#     var=vars[i]
#     plotvar=ip_ORCirr[var].mean(dim='time')
#     ax=axes[i]
#     cmap=cmaps[i]
#     vmin=vmins[i]
#     vmax=vmaxs[i]
#     clabel=clabels[i]
#     # ax.add_feature(cfeature.RIVERS)
#     nice_map(plotvar, ax, cmap, vmin, vmax, clabel=clabel)
#     title='({})'.format(letters[i])
#     ax.set_title(title)

# for i in range(2, 4):
#     var=vars[i]
#     rel_diff = (ip_ORCirr[var] - ip_ORCnoirr[var]).mean(dim='time') / ip_ORCnoirr[var].mean(dim='time') * 100
#     plotvar= rel_diff
#     ax=axes[i]
#     cmap=cmaps[i]
#     vmin=vmins[i]
#     vmax=vmaxs[i]
#     clabel=clabels[i]
#     nice_map(plotvar, ax, cmap, vmin, vmax, clabel=clabel)
#     #add rivers
#     # ax.add_feature(cfeature.RIVERS)
#     title='({})'.format(letters[i])
#     ax.set_title(title)

## Fig 4 : irrig eval vs obs_Ebro (Dari et al)

In [ ]:
var='irrigation'
ds1=obsEbro.sel(time=slice('2016-01-01','2020-07-31'))
ds2=ebro_ORCirr_iObsEbro.sel(time=slice('2016-01-01','2020-07-31'))

In [ ]:
#horizontal version
# fig = plt.figure(figsize=(18, 4.5))
# gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1])  

# # Seasonal cycle
# ax1 = fig.add_subplot(gs[0, 0])
# ylabel = 'Irrigation (mm d⁻¹)'

# plotvar1 = ds1[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
# color1 = 'black'
# label1 = 'obs_Ebro'
# nice_time_plot(plotvar1, ax1, label=label1, color=color1, ylabel=ylabel)

# plotvar2 = ds2[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
# color2 = 'blue'
# label2 = 'irr'
# nice_time_plot(plotvar2, ax1, label=label2, color=color2, ylabel=ylabel)

# ax1.set_xticks(np.arange(1, 13))
# ax1.set_xticklabels(months_name_list)
# ax1.set_title('(a)')

# # Map with PlateCarree projection
# ax2 = fig.add_subplot(gs[0, 1], projection=ccrs.PlateCarree()) 
# plotvar = ds2[var].mean(dim='time') - ds1[var].mean(dim='time')
# cmap = emb_neutral
# vmin = -0.5
# vmax = 0.5
# clabel = 'Irrigation bias (mm d⁻¹)'

# nice_map(plotvar, ax2, cmap, vmin, vmax, clabel=clabel)
# ax2.set_title('(b)')

# plt.tight_layout()

In [ ]:
#vertical version
fig = plt.figure(figsize=(8.5, 9))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 1])

# Seasonal cycle
ax1 = fig.add_subplot(gs[0, 0])  # Regular Cartesian plot
ylabel = 'Irrigation (mm d⁻¹)'

plotvar1 = ds1[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color1 = 'black'
label1 = 'Ebro_estimate'
nice_time_plot(plotvar1, ax1, label=label1, color=color1, ylabel=ylabel)

plotvar2 = ds2[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color2 = 'blue'
label2 = 'irr'
nice_time_plot(plotvar2, ax1, label=label2, color=color2, ylabel=ylabel)

ax1.set_xticks(np.arange(1, 13))
ax1.set_xticklabels(months_name_list)
ax1.set_title('(a)')

# Map with PlateCarree projection
ax2 = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree())
plotvar = (ds2[var] - ds1[var]).mean(dim='time')
cmap = emb_neutral
vmin = -0.5
vmax = 0.5
clabel = 'Irrigation bias (mm d⁻¹)'

nice_map(plotvar, ax2, cmap, vmin, vmax, clabel=clabel)
ax2.set_title('(b)')

plt.tight_layout()

## Fig 5 : 5 stations

In [ ]:
#plot a time series for each station from representative_stations_dict
fig, axes= plt.subplots(5,1, figsize=(10,25))
axes= axes.flatten()
ds_list=[routing_noirr, routing_irr]
for i, (station_id, station_data) in enumerate(representative_stations_dict.items()):
    title_letter=letters[i]
    # ts_station(obs_routing,axes[i], station_id, name=station_data['name'], ylabel='River discharge (m³/s)', year_min=2010, year_max=2022)
    # ts_with_obs(ds_list, obs_routing, axes[i], station_id, station_data, ylabel='River discharge (m³/s)', year_min=2010, year_max=2022)
    sc_station(obs_routing,axes[i], station_id, name=station_data['name'], ylabel='River discharge (m³ s⁻¹)', year_min=2010, year_max=2022)
    sc_with_obs(ds_list, obs_routing, axes[i], station_id, station_data, ylabel='River discharge (m³ s⁻¹)', year_min=2010, year_max=2022, title_letter=title_letter)

In [ ]:
# #plot a time series for each station from representative_stations_dict
# fig, axes= plt.subplots(2,2, figsize=(17,10))
# axes= axes.flatten()
# ds_list=[routing_noirr, routing_irr]
# for i, (station_id, station_data) in enumerate(representative_stations_dict.items()):
#     title_letter=letters[i]
#     # ts_station(obs_routing,axes[i], station_id, name=station_data['name'], ylabel='River discharge (m³/s)', year_min=2010, year_max=2022)
#     # ts_with_obs(ds_list, obs_routing, axes[i], station_id, station_data, ylabel='River discharge (m³/s)', year_min=2010, year_max=2022)
#     sc_station(obs_routing,axes[i], station_id, name=station_data['name'], ylabel='River discharge (m³ s⁻¹)', year_min=2010, year_max=2022)
#     sc_with_obs(ds_list, obs_routing, axes[i], station_id, station_data, ylabel='River discharge (m³ s⁻¹)', year_min=2010, year_max=2022, title_letter=title_letter)

## Fig 6 : ET and P eval vs GPCC and GLEAM

In [ ]:
fig = plt.figure(figsize=(18, 8))
gs = gridspec.GridSpec(2, 3, width_ratios=[1.1,1,1.2], height_ratios=[1, 1])

#precip
date_min = '2010-01-01'
date_max = '2019-12-31'
ds_obs=ip_gpcc_isim.sel(time=slice(date_min,date_max))
ds_irr=ip_simirr.sel(time=slice(date_min,date_max))
ds_noirr=ip_sim.sel(time=slice(date_min,date_max))
var='precip'
# Seasonal cycle
ax1 = fig.add_subplot(gs[0,0])
ylabel="Precipitation (mm d⁻¹)"
plotvar1 = ds_obs[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color1 = 'black'
label1 = 'GPCC'
nice_time_plot(plotvar1, ax1, vmin=0, label=label1, color=color1, ylabel=ylabel)
plotvar2 = ds_noirr[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color2 = 'red'
label2 = 'no_irr'
nice_time_plot(plotvar2, ax1, vmin=0, label=label2, color=color2, ylabel=ylabel)
plotvar3 = ds_irr[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color3 = 'blue'
label3 = 'irr'
nice_time_plot(plotvar3, ax1, vmin=0, vmax=3.5, label=label3, color=color3, ylabel=ylabel)
ax1.set_title('(a) Mean seasonnal cycle (2010-2019)')
ax1.set_xticks(np.arange(1, 13))
ax1.set_xticklabels(months_name_list)

#Diff
ax2 = fig.add_subplot(gs[0,1], projection=ccrs.PlateCarree())
plotvar=(ds_noirr[var]-ds_obs[var]).mean(dim='time')
cmap=emb_neutral
vmin=-2
vmax=2
clabel="Precipitation bias (mm d⁻¹)"
nice_map(plotvar, ax2, cmap, vmin, vmax, clabel=clabel, cbar_on=False)
ax2.set_title('(b) no_irr - GPCC')

ax3 = fig.add_subplot(gs[0,2], projection=ccrs.PlateCarree())
plotvar=(ds_irr[var]-ds_obs[var]).mean(dim='time')
cmap=emb_neutral
vmin=-2
vmax=2
clabel="Precipitation bias (mm d⁻¹)"
nice_map(plotvar, ax3, cmap, vmin, vmax, clabel=clabel, left_labels=False)
ax3.set_title('(c) irr - GPCC')


# #ET
date_min = '2010-01-01'
date_max = '2022-12-31'
ds_obs=ip_gleam_isim.sel(time=slice(date_min,date_max))
ds_irr=ip_simirr.sel(time=slice(date_min,date_max))
ds_noirr=ip_sim.sel(time=slice(date_min,date_max))
var='evap'
# Seasonal cycle
ax4 = fig.add_subplot(gs[1,0])
vmin=0
ylabel="Evapotranspiration (mm d⁻¹)"
plotvar1 = ds_obs[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color1 = 'black'
label1 = 'GLEAM'
nice_time_plot(plotvar1, ax4, vmin=0, label=label1, color=color1, ylabel=ylabel)
plotvar2 = ds_noirr[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color2 = 'red'
label2 = 'no_irr'
nice_time_plot(plotvar2, ax4, vmin=0, label=label2, color=color2, ylabel=ylabel)
plotvar3 = ds_irr[var].mean(dim=['lon', 'lat']).groupby('time.month').mean(dim='time')
color3 = 'blue'
label3 = 'irr'
nice_time_plot(plotvar3, ax4, vmin=0, vmax=2.5, label=label3, color=color3, ylabel=ylabel)
ax4.set_title('(d) Mean seasonnal cycle (2010-2022)')
ax4.set_xticks(np.arange(1, 13))
ax4.set_xticklabels(months_name_list)

#Diff
ax5 = fig.add_subplot(gs[1,1], projection=ccrs.PlateCarree())
plotvar=(ds_noirr[var]-ds_obs[var]).mean(dim='time')
cmap=emb_neutral
vmin=-1
vmax=1
clabel="Evapotranspiration bias (mm d⁻¹)"
nice_map(plotvar, ax5, cmap, vmin, vmax, clabel=clabel, cbar_on=False)
ax5.set_title('(e) no_irr - GLEAM')

ax6 = fig.add_subplot(gs[1,2], projection=ccrs.PlateCarree())
plotvar=(ds_irr[var]-ds_obs[var]).mean(dim='time')
cmap=emb_neutral
vmin=-1
vmax=1
clabel="Evapotranspiration bias (mm d⁻¹)"
nice_map(plotvar, ax6, cmap, vmin, vmax, clabel=clabel, left_labels=False)
ax6.set_title('(f) irr - GLEAM')

plt.tight_layout()

## Fig 7 : diffs irr - no_irr

In [ ]:
vars=['fluxlat', 'fluxsens', 'rh2m', 't2m', 's_lcl', 's_pblh', 'cape', 'precip']
vmins=[-50, -50, -5.5, -0.35, -250, -100, -35, -1]
vmaxs=[ 50,  50,  5.5,  0.35,  250,  100,  35,  1]
clabels=['Latent heat flux change (W m⁻²)',
         'Sensible heat flux change (W m⁻²)',
         '2-m relative humidity change (%)',
         '2-m temperature change (K)',
         'LCL change (m)',
         'ABL height change (m)',
         'CAPE change (J kg⁻¹)',
         'Precipitation change (mm d⁻¹)']
cmap=emb_neutral

fig, axes = plt.subplots(4, 2, figsize=(17,16), subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.flatten()


pvalue=0.05
sig_method=1
check_norm=False
sig_viz=6
hatch='///'

months=[6, 7, 8]
# months=[3,4,5]

for i in range(2):
    ds1=ip_ORCirr.sel(time=ip_ORCirr['time.month'].isin(months))
    ds2=ip_ORCnoirr.sel(time=ip_ORCnoirr['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap, vmin, vmax, sig_mask=sig_mask, clabel=clabel, hatch=hatch)
    #add rivers
    # ax.add_feature(cfeature.RIVERS)
    title='({})'.format(letters[i])
    ax.set_title(title)

for i in range(2, 8):
    ds1=ip_simirr.sel(time=ip_simirr['time.month'].isin(months))
    ds2=ip_sim.sel(time=ip_sim['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap, vmin, vmax, sig_mask=sig_mask, clabel=clabel, hatch=hatch)
    title='({})'.format(letters[i])
    ax.set_title(title)

In [ ]:
#define evaporative fraction
ip_ORCnoirr['evap_frac'] = ip_ORCnoirr['fluxlat'] / (ip_ORCnoirr['fluxlat'] + ip_ORCnoirr['fluxsens'])
ip_ORCirr['evap_frac'] = ip_ORCirr['fluxlat'] / (ip_ORCirr['fluxlat'] + ip_ORCirr['fluxsens'])
# add units
ip_ORCnoirr['evap_frac'].attrs['units']=''
ip_ORCirr['evap_frac'].attrs['units']=''

In [ ]:
#simplified version of the figure with only 4 or 6 maps
vars=['evap_frac', 't2m', 's_pblh', 's_lcl', 'precip', 'cape']
vmins=[-0.3, -0.35, -100, -250, -1, -35]
vmaxs=[ 0.3,  0.35,  100,  250,  1, 35]
clabels=['Evaporative fraction change',
         '2-m temperature change (K)',
         'ABL height change (m)',
         'LCL change (m)',
         'Precipitation change (mm d⁻¹)',
         'CAPE change (J kg⁻¹)']
cmap=emb_neutral

fig, axes = plt.subplots(3, 2, figsize=(17,12), subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.flatten()


pvalue=0.05
sig_method=1
check_norm=False
sig_viz=6
hatch='///'

months=[6, 7, 8]

for i in range(1):
    ds1=ip_ORCirr.sel(time=ip_ORCirr['time.month'].isin(months))
    ds2=ip_ORCnoirr.sel(time=ip_ORCnoirr['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap, vmin, vmax, sig_mask=sig_mask, clabel=clabel, hatch=hatch)
    title='({})'.format(letters[i])
    ax.set_title(title)

for i in range(1, 6):
    ds1=ip_simirr.sel(time=ip_simirr['time.month'].isin(months))
    ds2=ip_sim.sel(time=ip_sim['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap, vmin, vmax, sig_mask=sig_mask, clabel=clabel, hatch=hatch)
    title='({})'.format(letters[i])
    ax.set_title(title)

In [ ]:
#simplified version of the figure with only 4
vars=['evap_frac', 't2m', 's_pblh', 'precip']
vmins=[-0.3, -0.35, -100, -1]
vmaxs=[ 0.3,  0.35,  100,  1]
clabels=['Evaporative fraction change',
         '2-m temperature change (K)',
         'ABL height change (m)',
         'Precipitation change (mm d⁻¹)',
         ]
cmap=emb_neutral

fig, axes = plt.subplots(2, 2, figsize=(17,8), subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.flatten()


pvalue=0.05
sig_method=1
check_norm=False
sig_viz=6
hatch='///'

months=[6, 7, 8]

for i in range(1):
    ds1=ip_ORCirr.sel(time=ip_ORCirr['time.month'].isin(months))
    ds2=ip_ORCnoirr.sel(time=ip_ORCnoirr['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap, vmin, vmax, sig_mask=sig_mask, clabel=clabel, hatch=hatch)
    title='({})'.format(letters[i])
    ax.set_title(title)

for i in range(1, 4):
    ds1=ip_simirr.sel(time=ip_simirr['time.month'].isin(months))
    ds2=ip_sim.sel(time=ip_sim['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap, vmin, vmax, sig_mask=sig_mask, clabel=clabel, hatch=hatch)
    title='({})'.format(letters[i])
    ax.set_title(title)

In [ ]:
#With JJA irrig and wind
vars=['irrigation', 'evap_frac', 't2m', 's_pblh', 's_lcl', 'precip', 'cape', 'moisture_convergence']
vmins=[0  , -0.3, -0.35, -100, -250, -1, -35, -1]
vmaxs=[1.5,  0.3,  0.35,  100,  250,  1,  35,  1]
clabels=['Irrigation (mm d⁻¹)',
         'Evaporative fraction change',
         '2-m temperature change (K)',
         'ABL height change (m)',
         'LCL change (m)',
         'Precipitation change (mm d⁻¹)',
         'CAPE change (J kg⁻¹)', 
         'Convergence change (mm d⁻¹)']
cmap=emb_neutral

fig, axes = plt.subplots(4, 2, figsize=(17,16), subplot_kw={'projection': ccrs.PlateCarree()})
axes = axes.flatten()

months=[6, 7, 8]

i=0
map_wind(ip_simirr.sel(time=ip_simirr['time.month'].isin(months)), 
         axes[i], 
         extra_var=vars[i], 
         extra_ds=ip_ORCirr.sel(time=ip_ORCirr['time.month'].isin(months)),
         height='10m', dist=2, scale=30,
         cmap=wet, vmin=vmins[i],vmax=vmaxs[i], clabel=clabels[i])

pvalue=0.05
sig_method=1
check_norm=False
sig_viz=6
hatch='///'

for i in range(1,2):
    ds1=ip_ORCirr.sel(time=ip_ORCirr['time.month'].isin(months))
    ds2=ip_ORCnoirr.sel(time=ip_ORCnoirr['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap, vmin, vmax, sig_mask=sig_mask, clabel=clabel, hatch=hatch)


for i in range(2, 8):
    ds1=ip_simirr.sel(time=ip_simirr['time.month'].isin(months))
    ds2=ip_sim.sel(time=ip_sim['time.month'].isin(months))
    var=vars[i]
    diff = (ds1[var] - ds2[var]).mean(dim='time')
    plotvar= diff
    ax=axes[i]
    vmin=vmins[i]
    vmax=vmaxs[i]
    clabel=clabels[i]
    sig_mask = compute_sig_mask(ds1, ds2, var, check_norm=check_norm, method=sig_method, pvalue=pvalue)
    nice_map(plotvar, ax, cmap, vmin, vmax, sig_mask=sig_mask, clabel=clabel, hatch=hatch)
    

for i in range(8):
    title='({})'.format(letters[i])
    axes[i].set_title(title)

## Fig 8 : Scatter plot ET and P

In [ ]:
ds1=ip_ORCirr.mean(dim=['lon', 'lat'])
ds2=(ip_ORCirr - ip_ORCnoirr).mean(dim=['lon', 'lat'])

In [ ]:
fig, axes= plt.subplots(1,2, figsize=(18,6))
axes=axes.flatten()
var1='irrigation'
xlabel='Irrigation (mm d⁻¹)'
seasons=['MAM', 'JJA', 'SON']

#evap vs irrig
ax=axes[0]
var2='evap'
ylabel='Evapotranspiration change (mm d⁻¹)'
title='(a)'
scatter_vars_seasons_ax(ax,ds1, ds2, var1, var2, reg=True, plot_one=True, title=title, coloring=True,
                      is_1D=True, seasons_list=seasons, xlabel=xlabel, ylabel=ylabel)
#precip vs irrig
ax=axes[1]
var2='rain'
ylabel='Precipitation change (mm d⁻¹)'
title='(b)'
scatter_vars_seasons_ax(ax,ds1, ds2, var1, var2, reg=True, plot_one=True, title=title, coloring=True,
                      is_1D=True, seasons_list=seasons, xlabel=xlabel, ylabel=ylabel)

## Fig 9 : Moisture budget by zone

### Define datasets and masks

In [ ]:
# Define 2D masks
# Applied irrigation
irr_mask_low    = ip_ORCirr.mean(dim='time')['irrigation']<0.06
irr_mask_high   = ip_ORCirr.mean(dim='time')['irrigation']>0.12
irr_mask_med    = (~irr_mask_low) * (~irr_mask_high) * ip_mask

In [ ]:
# Define datasets orc
lowirr_ip_ORCnoirr = apply_2Dmask_to_dataset(ip_ORCnoirr, irr_mask_low, dsname='lowirr_noirr')
lowirr_ip_ORCirr = apply_2Dmask_to_dataset(ip_ORCirr, irr_mask_low, dsname='lowirr_irr')

medirr_ip_ORCnoirr = apply_2Dmask_to_dataset(ip_ORCnoirr, irr_mask_med, dsname='medirr_noirr')
medirr_ip_ORCirr = apply_2Dmask_to_dataset(ip_ORCirr, irr_mask_med, dsname='medirr_irr')

highirr_ip_ORCnoirr = apply_2Dmask_to_dataset(ip_ORCnoirr, irr_mask_high, dsname='highirr_noirr')
highirr_ip_ORCirr = apply_2Dmask_to_dataset(ip_ORCirr, irr_mask_high, dsname='highirr_irr')

In [ ]:
# Temporal and spatial mean 
# for IP
ip_ORCnoirr_mean = ip_ORCnoirr.mean(dim=['time','lon','lat'])
ip_ORCirr_mean = ip_ORCirr.mean(dim=['time','lon','lat'])

#by irrig class
lowirr_ip_ORCnoirr_mean     = lowirr_ip_ORCnoirr.mean(dim=['time','lon','lat'])
lowirr_ip_ORCirr_mean       = lowirr_ip_ORCirr.mean(dim=['time','lon','lat'])
medirr_ip_ORCnoirr_mean     = medirr_ip_ORCnoirr.mean(dim=['time','lon','lat'])
medirr_ip_ORCirr_mean       = medirr_ip_ORCirr.mean(dim=['time','lon','lat'])
highirr_ip_ORCnoirr_mean    = highirr_ip_ORCnoirr.mean(dim=['time','lon','lat'])
highirr_ip_ORCirr_mean      = highirr_ip_ORCirr.mean(dim=['time','lon','lat'])

In [ ]:
#diff
#over whole IP
ip_orc_diff_mean = ip_ORCirr_mean - ip_ORCnoirr_mean

#by irrig class
lowirr_ip_orc_diff_mean     = lowirr_ip_ORCirr_mean - lowirr_ip_ORCnoirr_mean
medirr_ip_orc_diff_mean     = medirr_ip_ORCirr_mean - medirr_ip_ORCnoirr_mean
highirr_ip_orc_diff_mean    = highirr_ip_ORCirr_mean -  highirr_ip_ORCnoirr_mean

### Compute and display data

In [ ]:
compute_dict=False

In [ ]:
if compute_dict:
    data_dict = {
        "Variable": ["Irrigation",  "ET change","P change"],
        "Low irrigation":               [0,0,0],
        "Medium irrigation":            [0,0,0],
        "High irrigation":              [0,0,0],
        "Iberian Peninsula":            [0,0,0]
    }
    # add average values for irrigation, ET chang and P change to dictionary
    data_dict['Low irrigation'][0] = lowirr_ip_ORCirr_mean['irrigation'].compute().item()
    data_dict['Low irrigation'][1] = lowirr_ip_orc_diff_mean['evap'].compute().item()
    data_dict['Low irrigation'][2] = lowirr_ip_orc_diff_mean['rain'].compute().item()

    data_dict['Medium irrigation'][0] = medirr_ip_ORCirr_mean['irrigation'].compute().item()
    data_dict['Medium irrigation'][1] = medirr_ip_orc_diff_mean['evap'].compute().item()
    data_dict['Medium irrigation'][2] = medirr_ip_orc_diff_mean['rain'].compute().item()

    data_dict['High irrigation'][0] = highirr_ip_ORCirr_mean['irrigation'].compute().item()
    data_dict['High irrigation'][1] = highirr_ip_orc_diff_mean['evap'].compute().item()
    data_dict['High irrigation'][2] = highirr_ip_orc_diff_mean['rain'].compute().item()

    data_dict['Iberian Peninsula'][0] = ip_ORCirr_mean['irrigation'].compute().item()
    data_dict['Iberian Peninsula'][1] = ip_orc_diff_mean['evap'].compute().item()
    data_dict['Iberian Peninsula'][2] = ip_orc_diff_mean['rain'].compute().item()

    print(data_dict)
    
else:
    #variables computed for each subdataset
    data_dict = {'Variable': ['Irrigation', 'ET change', 'P change'],
                 'Low irrigation': [0.03250407055020332, 0.04089963436126709, 0.022051572799682617],
                 'Medium irrigation': [0.08199943602085114, 0.09069693088531494, 0.010108709335327148],
                 'High irrigation': [0.21001876890659332, 0.2211529016494751, 0.009575486183166504],
                 'Iberian Peninsula': [0.06673634797334671, 0.07550418376922607, 0.0167844295501709]
                 }


In [ ]:
0.22051572799682617/0.09575486183166504

In [ ]:
#turn dict to df
irrZone_df = pd.DataFrame(data_dict)
# irrZone_df = irrZone_df.set_index('Variable')
irrZone_df

In [ ]:
irrZone_df_peryear=irrZone_df.copy()
irrZone_df_peryear['(a) Low irrigation'] = irrZone_df['Low irrigation'] * 365
irrZone_df_peryear['(b) Medium irrigation'] = irrZone_df['Medium irrigation'] * 365
irrZone_df_peryear['(c) High irrigation'] = irrZone_df['High irrigation'] * 365
irrZone_df_peryear['(d) Iberian Peninsula'] = irrZone_df['Iberian Peninsula'] * 365
#drop 4 initial columns
irrZone_df_peryear.drop(columns=irrZone_df.columns[1:5], inplace=True)

irrZone_df_peryear

In [ ]:
plot_side_by_side_barplots(irrZone_df_peryear)

# Tables

## Table 1 : gridded products
Not made using Python

## Tab 2 : discharge stations

In [ ]:
#make csv from proper_stations_dict
filename='../../../obs/stations_data.csv'
df = pd.DataFrame(proper_stations_dict).T
df.to_csv(filename)

## Tab 3 : Change in evaluated metrics

In [ ]:
#compute all metrics on one station (necessary for later automatic use of metrics)
station_id, station = next(iter(proper_stations_dict.items()))
metric_list=[metric_sim_module, metric_obs_module, metric_bias, metric_rmse, metric_tcorr, metric_nse, metric_kge]
for metric_to_use in metric_list:
    metric_value=compute_metric_station(routing_noirr, obs_routing, station_id, station, metric_to_use)
    print('{} for station {} : {}'.format(metric_to_use.__short_name__, station['name'], metric_value))

In [ ]:
# output a pandas dataframe with all metric values for a given list of metrics and stations
sim=routing_noirr
metric_list=[metric_sim_module, metric_obs_module, metric_bias, metric_rmse, metric_tcorr, metric_nse, metric_kge]
stations_dict=proper_stations_dict
# define dataframe with one row per station and one column per metric
df=[]
for station_id, station in stations_dict.items():
    label='{} ({})'.format(station['station_nb'],station['name'])
    df.append({'Station':label})
    for metric in metric_list:
        name=metric.__short_name__
        metric_value=compute_metric_station(sim, obs_routing, station_id, station, metric)
        metric_value=np.round(metric_value, 2)
        # append metric_value to df
        df[-1][name]=metric_value

# convert df to pandas dataframe
df_noirr=pd.DataFrame(df)
# set Station as index
df_noirr.set_index('Station', inplace=True)
df_noirr['Bias (%)'] = np.round(df_noirr['Bias (m³/s)'] / df_noirr['Module (obs, m³/s)'], 3) * 100
#move Bias (%) to the 4th column
cols = df_noirr.columns.tolist()
cols = cols[:3] + cols[-1:] + cols[3:-1]
df_noirr = df_noirr[cols]
#add average row
df_noirr.loc['Mean'] = df_noirr.mean()
df_noirr.drop('Module (sim, m³/s)', axis=1, inplace=False)

In [ ]:
# output a pandas dataframe with all metric values for a given list of metrics and stations
sim=routing_irr
metric_list=[metric_sim_module, metric_obs_module, metric_bias, metric_rmse, metric_tcorr, metric_nse, metric_kge]
stations_dict=proper_stations_dict
# define dataframe with one row per station and one column per metric
df=[]
for station_id, station in stations_dict.items():
    label='{} ({})'.format(station['station_nb'],station['name'])
    df.append({'Station':label})
    for metric in metric_list:
        name=metric.__short_name__
        metric_value=compute_metric_station(sim, obs_routing, station_id, station, metric)
        metric_value=np.round(metric_value, 2)
        # append metric_value to df
        df[-1][name]=metric_value

# convert df to pandas dataframe
df_irr=pd.DataFrame(df)
# set Station as index
df_irr.set_index('Station', inplace=True)
df_irr['Bias (%)'] = np.round(df_irr['Bias (m³/s)'] / df_irr['Module (obs, m³/s)'], 3) * 100
#move Bias (%) to the 4th column
cols = df_irr.columns.tolist()
cols = cols[:3] + cols[-1:] + cols[3:-1]
df_irr = df_irr[cols]
#add average row
df_irr.loc['Mean'] = df_irr.mean()
df_irr.drop('Module (obs, m³/s)', axis=1, inplace=False).drop('Module (sim, m³/s)', axis=1, inplace=False)

In [ ]:
df_diff=df_irr-df_noirr
df_diff['Bias change (%)'] = np.round(df_diff['Bias (m³/s)'] / df_noirr['Bias (m³/s)'], 3) * 100
df_diff['RMSE change (%)'] = np.round(df_diff['RMSE'] / df_noirr['RMSE'], 4) * 100
df_diff.drop('Module (obs, m³/s)', axis=1, inplace=True)
df_diff.drop('Module (sim, m³/s)', axis=1, inplace=True)
df_diff.drop('Bias (m³/s)', axis=1, inplace=True)
df_diff.drop('Bias (%)', axis=1, inplace=True)
df_diff.drop('RMSE', axis=1, inplace=True)
# df_diff.loc['Mean'] = df_diff.mean()
df_diff

In [ ]:
#export to csv to in latex
df_noirr.to_csv('figures/df_noirr.csv')
df_irr.to_csv('figures/df_irr.csv')
df_diff.to_csv('figures/df_diff.csv')

## Tab 4 : Subdomains of different irrigation intensity


Areas (computed here) and mean irrigation (see Fig.9)

Thresholds defined in Fig.9 as well, 0.05 mm/d and 0.1 mm/d

```
irr_mask_low    = ip_ORCirr.mean(dim='time')['irrigation']<0.06
irr_mask_high   = ip_ORCirr.mean(dim='time')['irrigation']>0.12
irr_mask_med    = (~irr_mask_low) * (~irr_mask_high) * ip_mask
```

In [ ]:
ds_area= compute_grid_cell_width(ip_ORCirr)

mask1=irr_mask_low
mask_area_low=compute_mask_area(ds_area, mask1)

mask2=irr_mask_med
mask_area_med=compute_mask_area(ds_area, mask2)

mask3=irr_mask_high
mask_area_high=compute_mask_area(ds_area, mask3)

# Supplementary

## 18 stations time series (S1) and seasonal cycle (S2)

In [ ]:
#plot a time series for each station from dict
fig, axes= plt.subplots(6,3, figsize=(20,30))
axes= axes.flatten()
ds_list=[routing_noirr, routing_irr]
for i, (station_id, station_data) in enumerate(proper_stations_dict.items()):
    ts_station(obs_routing,axes[i], station_id, name=station_data['name'], year_min=2010, year_max=2022)
    ts_with_obs(ds_list, obs_routing, axes[i], station_id, station_data, year_min=2010, year_max=2022)

In [ ]:
#plot a time series for each station from dict
fig, axes= plt.subplots(6,3, figsize=(20,30))
axes= axes.flatten()
ds_list=[routing_noirr, routing_irr]
for i, (station_id, station_data) in enumerate(proper_stations_dict.items()):
    sc_station(obs_routing,axes[i], station_id, name=station_data['name'], year_min=2010, year_max=2022)
    sc_with_obs(ds_list, obs_routing, axes[i], station_id, station_data, year_min=2010, year_max=2022)